In [1]:
import os
import re
import shutil
from tqdm import tqdm
from PIL import Image

folder_name = 'Sprite'
files = os.listdir(folder_name)
skill_regex = r'^(Skill.*|^MonsterSkill.*)_HD.png'
status_regex = r'^UI_Gcg_Buff_(.*).png'
cardface_regex = r'^UI_Gcg_CardFace_((Assist_[^_]*_|Char_[^_]*_|Event_[^_]*_|Modify_[^_]*_|Summon).*).png$'
small_card_regex = r'^UI_Gcg_History_(.*)_S.png$'
avatar_regex = r'^UI_Gcg_Char_.*Icon_(.*).png'
tag_regex = r'^UI_Gcg_Tag_(.*).png'
mark_regex = r'^UI_TeyvatCard_((LifeBg|Preview|Revive|Select|Death).*).png'
dice_regex = r'UI_Gcg_DiceS_(.*).png'

In [2]:
keys = [x for x in globals().keys() if re.match(r'^.*_regex$', x)]
keys

['skill_regex',
 'status_regex',
 'cardface_regex',
 'small_card_regex',
 'avatar_regex',
 'tag_regex',
 'mark_regex',
 'dice_regex']

In [3]:
keys = ['avatar_regex']
for key in keys:
    fname = key.replace('_regex', '')
    regex = globals()[key]
    if not os.path.exists(fname):
        os.mkdir(fname)
    counter = 0
    for file in files:
        m = re.match(regex, file)
        if m:
            newname = m.group(1)
            if key == 'cardface_regex':
                # if is cardface, need more check

                # if has golden, skip
                if 'Golden' in file:
                    continue
                # if name contains #number, remove it from name
                newname = re.sub(r'#\d+', '', newname)
                newname = re.sub(r'_Layer\d+', '', newname)
                newname += '.png'
                if os.path.exists(os.path.join(fname, newname)):
                    # exists, check both image size
                    this_size = Image.open(os.path.join(folder_name, file)).size
                    target_size = Image.open(os.path.join(fname, newname)).size
                    if this_size[0] < target_size[0]:
                        # this is smaller, skip
                        continue
            else:
                newname += '.png'
            # newname = file
            # copy file to folder
            shutil.copy(os.path.join(folder_name, file), os.path.join(fname, newname))
            counter += 1
    print(key, counter)

avatar_regex 93


In [27]:
# check img size of cardface
for file in os.listdir('cardface'):
    size = Image.open(os.path.join('cardface', file)).size
    if size[0] != 420:
        print(file, size)
# check file prefix
prefix = set()
for file in os.listdir('cardface'):
    prefix.add('_'.join(file.split('_')[:-1]))
for p in prefix:
    print(p)

Char_Avatar
Modify_Talent
Assist_NPC
Summon_AbyssEle
Event_Event
Char_Monster
Assist_Prop
Event_Food
Event_ResonanceB
Modify_Constellation
Modify_Artifact
Modify_Weapon
Event_ResonanceA
Assist_Location
Summon
Char_Enemy


In [30]:
# move Texture2D files to mark
files = os.listdir('Texture2D')
for file in files:
    newname = file[:-4]
    newname = newname.replace('_Bg', '')
    newname = newname.split('_')[-1]
    shutil.copy(os.path.join('Texture2D', file), os.path.join('mark', newname + '.png'))

In [ ]:
# check small_card all exist in cardface
small_card = set(os.listdir('small_card'))
cardface = set(os.listdir('cardface'))
assert small_card.issubset(cardface)